<div style="clear: both; width: 100%; overflow: auto"><img src="img/yabox.png" style="width: 250px; float: left"/></div>

> Yabox: Yet another black-box optimization library for Python - https://github.com/pablormier/yabox

Tutorial of the basic functionality of Yabox

Author: Pablo Rodríguez-Mier, [@pablormier](https://twitter.com/PabloRMier)

# Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys

# Load Yabox (from local)
# Comment this line to use the installed version
sys.path.insert(0, '../')

import yabox as yb

# Import the DE implementations
from yabox.algorithms import DE, PDE
print('Yabox version: ', yb.__version__)

# Basics

Optimization of a lambda function:

In [ ]:
DE(lambda x: sum(x**2), [(-10, 10)], maxiters=1000).solve(show_progress=True)

In [ ]:
def fun(x, a, b):
    return a + sum(x**2) / (b + 1)

def f(x):
    # You can initialize here your params a, b
    return fun(x, 10, -2)

DE(f, [(-10, 10)], maxiters=1000).solve(show_progress=True)

You can control also the search process. Use `geniterator()` to get an iterator at the level of the population (by default, `.iterator()` returns an iterator at the level of individual (thus it iteraters maxiters * popsize  times)

In [ ]:
# Control the iteration process
de = DE(lambda x: sum(x**2), [(-10, 10)], maxiters=30)
for step in de.geniterator():
    idx = step.best_idx
    norm_vector = step.population[idx]
    best_params = de.denormalize([norm_vector])
    print(step.best_fitness, norm_vector, best_params[0])

In [ ]:
# NOTE: On windows, processes are not spawn from ipython
def f(x):
    return sum(x**2)

PDE(f, [(-10, 10)], maxiters=1000).solve(show_progress=False)